# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load dnsga2 results for all topos

In [2]:
with open('res_bc_Cogent_dnsga2.pkl','rb') as f_Cogent:
    res_11_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_dnsga2.pkl','rb') as f_UsCarrier:
    res_11_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_dnsga2.pkl','rb') as f_HiberniaGlobal:
    res_11_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_dnsga2.pkl','rb') as f_Colt:
    res_11_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_dnsga2.pkl','rb') as f_Funet:
    res_11_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_dnsga2.pkl','rb') as f_Abvt:
    res_11_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_dnsga2.pkl','rb') as f_Intellifiber:
    res_11_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_dnsga2.pkl','rb') as f_TataNld:
    res_11_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_dnsga2.pkl','rb') as f_Internode:
    res_11_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_dnsga2.pkl','rb') as f_Missouri:
    res_11_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_dnsga2.pkl','rb') as f_Ion:
    res_11_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_dnsga2.pkl','rb') as f_Palmetto:
    res_11_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:
# dnsga2
F11_Cogent=res_11_Cogent.F
F11_UsCarrier=res_11_UsCarrier.F
F11_HiberniaGlobal=res_11_HiberniaGlobal.F
F11_Colt=res_11_Colt.F
F11_Funet=res_11_Funet.F
F11_Abvt=res_11_Abvt.F
F11_Intellifiber=res_11_Intellifiber.F
F11_TataNld=res_11_TataNld.F
F11_Internode=res_11_Internode.F
F11_Missouri=res_11_Missouri.F
F11_Ion=res_11_Ion.F
F11_Palmetto=res_11_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F11_Cogent = []
hist_F11_UsCarrier = []
hist_F11_HiberniaGlobal = []
hist_F11_Colt = []
hist_F11_Funet = []
hist_F11_Abvt = []
hist_F11_Intellifiber = []
hist_F11_TataNld = []
hist_F11_Internode = []
hist_F11_Missouri = []
hist_F11_Ion = []
hist_F11_Palmetto = []


for algo11_Cogent in res_11_Cogent.history:
    opt11_Cogent = algo11_Cogent.opt
    feas11_Cogent = np.where(opt11_Cogent.get("feasible"))[0]
    hist_F11_Cogent.append(opt11_Cogent.get("F")[feas11_Cogent])
for algo11_UsCarrier in res_11_UsCarrier.history:
    opt11_UsCarrier = algo11_UsCarrier.opt
    feas11_UsCarrier = np.where(opt11_UsCarrier.get("feasible"))[0]
    hist_F11_UsCarrier.append(opt11_UsCarrier.get("F")[feas11_UsCarrier])
for algo11_HiberniaGlobal in res_11_HiberniaGlobal.history:
    opt11_HiberniaGlobal = algo11_HiberniaGlobal.opt
    feas11_HiberniaGlobal = np.where(opt11_HiberniaGlobal.get("feasible"))[0]
    hist_F11_HiberniaGlobal.append(opt11_HiberniaGlobal.get("F")[feas11_HiberniaGlobal])
for algo11_Colt in res_11_Colt.history:
    opt11_Colt = algo11_Colt.opt
    feas11_Colt = np.where(opt11_Colt.get("feasible"))[0]
    hist_F11_Colt.append(opt11_Colt.get("F")[feas11_Colt])
for algo11_Funet in res_11_Funet.history:
    opt11_Funet = algo11_Funet.opt
    feas11_Funet = np.where(opt11_Funet.get("feasible"))[0]
    hist_F11_Funet.append(opt11_Funet.get("F")[feas11_Funet])
for algo11_Abvt in res_11_Abvt.history:
    opt11_Abvt = algo11_Abvt.opt
    feas11_Abvt = np.where(opt11_Abvt.get("feasible"))[0]
    hist_F11_Abvt.append(opt11_Abvt.get("F")[feas11_Abvt])
for algo11_Intellifiber in res_11_Intellifiber.history:
    opt11_Intellifiber = algo11_Intellifiber.opt
    feas11_Intellifiber = np.where(opt11_Intellifiber.get("feasible"))[0]
    hist_F11_Intellifiber.append(opt11_Intellifiber.get("F")[feas11_Intellifiber])
for algo11_TataNld in res_11_TataNld.history:
    opt11_TataNld = algo11_TataNld.opt
    feas11_TataNld = np.where(opt11_TataNld.get("feasible"))[0]
    hist_F11_TataNld.append(opt11_TataNld.get("F")[feas11_TataNld])
for algo11_Internode in res_11_Internode.history:
    opt11_Internode = algo11_Internode.opt
    feas11_Internode= np.where(opt11_Internode.get("feasible"))[0]
    hist_F11_Internode.append(opt11_Internode.get("F")[feas11_Internode])
for algo11_Missouri in res_11_Missouri.history:
    opt11_Missouri = algo11_Missouri.opt
    feas11_Missouri = np.where(opt11_Missouri.get("feasible"))[0]
    hist_F11_Missouri.append(opt11_Missouri.get("F")[feas11_Missouri])
for algo11_Ion in res_11_Ion.history:
    opt11_Ion = algo11_Ion.opt
    feas11_Ion = np.where(opt11_Ion.get("feasible"))[0]
    hist_F11_Ion.append(opt11_Ion.get("F")[feas11_Ion])
for algo11_Palmetto in res_11_Palmetto.history:
    opt11_Palmetto = algo11_Palmetto.opt
    feas11_Palmetto = np.where(opt11_Palmetto.get("feasible"))[0]
    hist_F11_Palmetto.append(opt11_Palmetto.get("F")[feas11_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for dnsga2 algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_11_Cogent = [metric.do(_F11_Cogent) for _F11_Cogent in hist_F11_Cogent]
hv_11_UsCarrier = [metric.do(_F11_UsCarrier) for _F11_UsCarrier in hist_F11_UsCarrier]
hv_11_HiberniaGlobal = [metric.do(_F11_HiberniaGlobal) for _F11_HiberniaGlobal in hist_F11_HiberniaGlobal]
hv_11_Colt = [metric.do(_F11_Colt) for _F11_Colt in hist_F11_Colt]
hv_11_Funet = [metric.do(_F11_Funet) for _F11_Funet in hist_F11_Funet]
hv_11_Abvt = [metric.do(_F11_Abvt) for _F11_Abvt in hist_F11_Abvt]
hv_11_Intellifiber = [metric.do(_F11_Intellifiber) for _F11_Intellifiber in hist_F11_Intellifiber]
hv_11_TataNld = [metric.do(_F11_TataNld) for _F11_TataNld in hist_F11_TataNld]
hv_11_Internode = [metric.do(_F11_Internode) for _F11_Internode in hist_F11_Internode]
hv_11_Missouri = [metric.do(_F11_Missouri) for _F11_Missouri in hist_F11_Missouri]
hv_11_Ion = [metric.do(_F11_Ion) for _F11_Ion in hist_F11_Ion]
hv_11_Palmetto = [metric.do(_F11_Palmetto) for _F11_Palmetto in hist_F11_Palmetto]

# dnsga2
hv_11 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_11_Cogent, hv_11_UsCarrier, hv_11_HiberniaGlobal, hv_11_Colt, hv_11_Funet, hv_11_Abvt, hv_11_Intellifiber, hv_11_TataNld, hv_11_Internode, hv_11_Missouri, hv_11_Ion, hv_11_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_11_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_11_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_11_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_11_Colt)
print('The hypervolumn for Funet:')
print(hv_11_Funet)
print('The hypervolumn for Abvt:')
print(hv_11_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_11_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_11_TataNld)
print('The hypervolumn for Internode:')
print(hv_11_Internode)
print('The hypervolumn for Missouri:')
print(hv_11_Missouri)
print('The hypervolumn for Ion:')
print(hv_11_Ion)
print('The hypervolumn for Palmetto:')
print(hv_11_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_11_Cogent)+1)), hv_11_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_11_UsCarrier)+1)), hv_11_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_11_HiberniaGlobal)+1)), hv_11_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_11_Colt)+1)), hv_11_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_11_Funet)+1)), hv_11_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_11_Abvt)+1)), hv_11_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_11_Intellifiber)+1)), hv_11_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_11_TataNld)+1)), hv_11_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_11_Internode)+1)), hv_11_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_11_Missouri)+1)), hv_11_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_11_Ion)+1)), hv_11_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_11_Palmetto)+1)), hv_11_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for dnsga2:')
print(hv_11)

3. Draw mean hypervolume for dnsga2

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
# plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()